## Load the benchmark and view one example from the benchmark

In [1]:
api_key = input("Enter your OpenAI API key: ")
import dspy
lm = dspy.LM("openai/gpt-4.1-nano", temperature=1, api_key=api_key)
dspy.configure(lm=lm)

In [4]:
import requests
import dspy
import json
import os
import random

def init_dataset():
    # Load from the url
    url = "https://raw.githubusercontent.com/meta-llama/llama-prompt-ops/refs/heads/main/use-cases/facility-support-analyzer/dataset.json"
    dataset = json.loads(requests.get(url).text)
    dspy_dataset = [
        dspy.Example({
            "message": d['fields']['input'],
            "answer": d['answer'],
        }).with_inputs("message")
        for d in dataset
    ]
    random.Random(0).shuffle(dspy_dataset)
    train_set = dspy_dataset[:int(len(dspy_dataset) * 0.33)]
    val_set = dspy_dataset[int(len(dspy_dataset) * 0.33):int(len(dspy_dataset) * 0.66)]
    test_set = dspy_dataset[int(len(dspy_dataset) * 0.66):]

    return train_set, val_set, test_set

In [5]:
train_set, val_set, test_set = init_dataset()

len(train_set), len(val_set), len(test_set)

(66, 66, 68)

Let's view an example task input

In [25]:
print("Input Message:")
print(train_set[0]['message'])

print("\n\nGold Answer:")
for k, v in json.loads(train_set[0]['answer']).items():
    print(f"{k}: {v}")

Input Message:
Subject: Adjusting Bi-Weekly Cleaning Schedule for My Office

Dear ProCare Facility Solutions Support Team,

I hope this message finds you well. My name is Dr. Alex Turner, and I have been utilizing your services for my office space for the past year. I must say, your team's dedication to maintaining a pristine environment has been commendable and greatly appreciated.

I am reaching out to discuss the scheduling of our regular cleaning services. While I find the logistical challenges of coordinating these services intellectually stimulating, I believe we could optimize the current schedule to better suit the needs of my team and our workflow. Specifically, I would like to explore the possibility of adjusting our cleaning schedule to a bi-weekly arrangement, ideally on Tuesdays and Fridays, to ensure our workspace remains consistently clean without disrupting our research activities.

Previously, I have attempted to adjust the schedule through the online portal, but I enc

## Let's define a simple program to solve this task
The program is a 3-module system, each of which handles the urgency, sentiment and categories classification respectively

In [27]:
from typing import List, Literal


class FacilitySupportAnalyzerUrgency(dspy.Signature):
    """
    Read the provided message and determine the urgency.
    """
    message: str = dspy.InputField()
    urgency: Literal['low', 'medium', 'high'] = dspy.OutputField()

class FacilitySupportAnalyzerSentiment(dspy.Signature):
    """
    Read the provided message and determine the sentiment.
    """
    message: str = dspy.InputField()
    sentiment: Literal['positive', 'neutral', 'negative'] = dspy.OutputField()

class FacilitySupportAnalyzerCategories(dspy.Signature):
    """
    Read the provided message and determine the set of categories applicable to the message.
    """
    message: str = dspy.InputField()
    categories: List[Literal["emergency_repair_services", "routine_maintenance_requests", "quality_and_safety_concerns", "specialized_cleaning_services", "general_inquiries", "sustainability_and_environmental_practices", "training_and_support_requests", "cleaning_services_scheduling", "customer_feedback_and_complaints", "facility_management_issues"]] = dspy.OutputField()

class FacilitySupportAnalyzerMM(dspy.Module):
    def __init__(self):
        self.urgency_module = dspy.ChainOfThought(FacilitySupportAnalyzerUrgency)
        self.sentiment_module = dspy.ChainOfThought(FacilitySupportAnalyzerSentiment)
        self.categories_module = dspy.ChainOfThought(FacilitySupportAnalyzerCategories)
    
    def __call__(self, message: str):
        urgency = self.urgency_module(message=message)
        sentiment = self.sentiment_module(message=message)
        categories = self.categories_module(message=message)

        return dspy.Prediction(
            urgency=urgency.urgency,
            sentiment=sentiment.sentiment,
            categories=categories.categories
        )

program = FacilitySupportAnalyzerMM()

In [33]:
def score_urgency(gold_urgency, pred_urgency):
    """
    Compute score for the urgency module.
    """
    score = 1.0 if gold_urgency == pred_urgency else 0.0
    return score

def score_sentiment(gold_sentiment, pred_sentiment):
    """
    Compute score for the sentiment module.
    """
    score = 1.0 if gold_sentiment == pred_sentiment else 0.0
    return score

def score_categories(gold_categories, pred_categories):
    """
    Compute score for the categories module.
    Uses the same match/mismatch logic as category accuracy in the score.
    """
    correct = 0
    for k, v in gold_categories.items():
        if v and k in pred_categories:
            correct += 1
        elif not v and k not in pred_categories:
            correct += 1
    score = correct / len(gold_categories)
    return score

def metric(example, pred, trace=None, pred_name=None, pred_trace=None):
    """
    Computes a score based on agreement between prediction and gold standard for categories, sentiment, and urgency.
    Returns the score (float).
    """
    # Parse gold standard from example
    gold = json.loads(example['answer'])

    # Compute scores for all modules
    score_urgency_val = score_urgency(gold['urgency'], pred.urgency)
    score_sentiment_val = score_sentiment(gold['sentiment'], pred.sentiment)
    score_categories_val = score_categories(gold['categories'], pred.categories)

    # Overall score: average of the three accuracies
    total = (score_urgency_val + score_sentiment_val + score_categories_val) / 3

    return total

## Define an evaluator and evaluate the base program

In [34]:
import dspy
evaluate = dspy.Evaluate(
    devset=test_set,
    metric=metric,
    num_threads=32,
    display_table=True,
    display_progress=True
)

evaluate(program)

Average Metric: 51.30 / 68 (75.4%): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [00:00<00:00, 322.00it/s]

2025/08/12 18:09:18 INFO dspy.evaluate.evaluate: Average Metric: 51.3 / 68 (75.4%)


,message,answer,urgency,sentiment,categories,metric
0,"Hey ProCare Support Team, Hope you all are doing great! My name is...","{""categories"": {""routine_maintenance_requests"": false, ""customer_f...",low,positive,[sustainability_and_environmental_practices],✔️ [1.000]
1,"Hey ProCare Team, Hope you’re all doing well! My name’s Jake, and ...","{""categories"": {""routine_maintenance_requests"": true, ""customer_fe...",medium,positive,"[routine_maintenance_requests, customer_feedback_and_complaints]",✔️ [0.967]
2,"Subject: Assistance Needed for HVAC Maintenance Hi [Receiver], I h...","{""categories"": {""routine_maintenance_requests"": true, ""customer_fe...",medium,neutral,[routine_maintenance_requests],✔️ [1.000]
3,Subject: A Green Inquiry from a Bill Maher Enthusiast Hey ProCare ...,"{""categories"": {""routine_maintenance_requests"": false, ""customer_f...",low,positive,[sustainability_and_environmental_practices],✔️ [1.000]
4,Subject: Inquiry on Sustainability Practices Dear ProCare Facility...,"{""categories"": {""routine_maintenance_requests"": false, ""customer_f...",medium,neutral,[sustainability_and_environmental_practices],✔️ [0.667]
...,...,...,...,...,...,...
63,Subject: Inquiry About Your Eco-Friendly Practices Dear ProCare Fa...,"{""categories"": {""routine_maintenance_requests"": false, ""customer_f...",medium,neutral,[sustainability_and_environmental_practices],✔️ [0.600]
64,Subject: Assistance Needed for Facility Management Issue Dear ProC...,"{""categories"": {""routine_maintenance_requests"": false, ""customer_f...",high,positive,[facility_management_issues],✔️ [0.667]
65,"Subject: Request for Training and Support Hi ProCare Support Team,...","{""categories"": {""routine_maintenance_requests"": false, ""customer_f...",low,positive,[training_and_support_requests],✔️ [1.000]
66,Subject: Concerns About Studio Maintenance and Rent Increase Dear ...,"{""categories"": {""routine_maintenance_requests"": true, ""customer_fe...",medium,negative,"[routine_maintenance_requests, facility_management_issues]",✔️ [0.600]


EvaluationResult(score=75.44, results=<list of 68 results>)

## Load the GEPA Optimizer

In [45]:
import json
import dspy

def feedback_urgency(gold_urgency, pred_urgency):
    """
    Generate feedback for the urgency module.
    """
    score = 1.0 if gold_urgency == pred_urgency else 0.0
    if gold_urgency == pred_urgency:
        feedback = f"You correctly classified the urgency of the message as `{gold_urgency}`. This message is indeed of `{gold_urgency}` urgency."
    else:
        feedback = f"You incorrectly classified the urgency of the message as `{pred_urgency}`. The correct urgency is `{gold_urgency}`. Think about how you could have reasoned to get the correct urgency label."
    return feedback, score

def feedback_sentiment(gold_sentiment, pred_sentiment):
    """
    Generate feedback for the sentiment module.
    """
    score = 1.0 if gold_sentiment == pred_sentiment else 0.0
    if gold_sentiment == pred_sentiment:
        feedback = f"You correctly classified the sentiment of the message as `{gold_sentiment}`. This message is indeed `{gold_sentiment}`."
    else:
        feedback = f"You incorrectly classified the sentiment of the message as `{pred_sentiment}`. The correct sentiment is `{gold_sentiment}`. Think about how you could have reasoned to get the correct sentiment label."
    return feedback, score

def feedback_categories(gold_categories, pred_categories):
    """
    Generate feedback for the categories module.
    Uses the same match/mismatch logic as category accuracy in the score.
    """
    correctly_included = [k for k, v in gold_categories.items() if v and k in pred_categories]
    incorrectly_included = [k for k, v in gold_categories.items() if not v and k in pred_categories]
    incorrectly_excluded = [k for k, v in gold_categories.items() if v and k not in pred_categories]
    correctly_excluded = [k for k, v in gold_categories.items() if not v and k not in pred_categories]  # For completeness in accuracy check

    # Recompute category accuracy (matches score logic)
    score = (len(correctly_included) + len(correctly_excluded)) / len(gold_categories)

    if score == 1.0:
        fb_text = f"The category classification is perfect. You correctly identified that the message falls under the following categories: `{repr(correctly_included)}`."
    else:
        fb_text = f"The category classification is not perfect. You correctly identified that the message falls under the following categories: `{repr(correctly_included)}`.\n"
        if incorrectly_included:
            fb_text += f"However, you incorrectly identified that the message falls under the following categories: `{repr(incorrectly_included)}`. The message DOES NOT fall under these categories.\n"
        if incorrectly_excluded:
            prefix = "Additionally, " if incorrectly_included else "However, "
            fb_text += f"{prefix}you didn't identify the following categories that the message actually falls under: `{repr(incorrectly_excluded)}`.\n"
        fb_text += "Think about how you could have reasoned to get the correct category labels."
    return fb_text, score

def metric_with_feedback(example, pred, trace=None, pred_name=None, pred_trace=None):
    """
    Computes a score based on agreement between prediction and gold standard for categories, sentiment, and urgency.
    Optionally provides feedback text for a specific predictor module, using the same comparison logic as the score.
    Returns a dspy.Prediction with score (float) and feedback (str).
    """
    # Parse gold standard from example
    gold = json.loads(example['answer'])

    # Compute feedback and scores for all modules
    fb_urgency, score_urgency = feedback_urgency(gold['urgency'], pred.urgency)
    fb_sentiment, score_sentiment = feedback_sentiment(gold['sentiment'], pred.sentiment)
    fb_categories, score_categories = feedback_categories(gold['categories'], pred.categories)

    # Overall score: average of the three accuracies
    total = (score_urgency + score_sentiment + score_categories) / 3

    if pred_name is None:
        return total

    elif pred_name == 'urgency_module.predict':
        feedback = fb_urgency
    elif pred_name == 'sentiment_module.predict':
        feedback = fb_sentiment
    elif pred_name == 'categories_module.predict':
        feedback = fb_categories

    return dspy.Prediction(score=total, feedback=feedback)

In [48]:
# Import GEPA and define the optimizer

from dspy import GEPA

optimizer = GEPA(
    metric=metric_with_feedback,
    auto="light",
    num_threads=32,
    track_stats=True,
    reflection_lm=dspy.LM(model="gpt-5", temperature=1.0, max_tokens=32000, api_key=api_key)
)

## Optimize the program with GEPA

In [49]:
optimized_program = optimizer.compile(
    program,
    trainset=train_set,
    valset=val_set,
)

2025/08/12 18:13:46 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 1643 metric calls of the program. This amounts to 12.45 full evals on the train+val set.
2025/08/12 18:13:46 INFO dspy.teleprompt.gepa.gepa: Using 66 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget.


7


2025/08/12 18:13:48 INFO dspy.evaluate.evaluate: Average Metric: 47.56666666666666 / 66 (72.1%)
2025/08/12 18:13:48 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.7207070707070706
2025/08/12 18:13:48 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.7207070707070706


Average Metric: 2.27 / 3 (75.6%): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 58.24it/s]

2025/08/12 18:13:48 INFO dspy.evaluate.evaluate: Average Metric: 2.2666666666666666 / 3 (75.6%)
2025/08/12 18:13:48 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for urgency_module.predict: Task: Determine the urgency of a customer message to ProCare Facility Solutions.

Context and domain:
- Messages are typically sent to ProCare Facility Solutions’ support team about facilities services (e.g., office/residential maintenance, cleaning, HVAC).
- Common topics include cleaning quality (especially in high-traffic areas), HVAC performance/safety, routine maintenance scheduling, and general inquiries (e.g., sustainability practices).

How to assess urgency:
Use these primary factors:
1) Safety and risk:
   - High/urgent if there’s an immediate safety hazard or potential harm (e.g., electrical sparks, gas smell, active water leak/flood, critical HVAC failure in extreme conditions, security breach).
   - Medium if safety is mentioned but described as minor or without signs o

...

2025/08/12 18:39:16 INFO dspy.evaluate.evaluate: Average Metric: 1.9666666666666666 / 3 (65.6%)


2025/08/12 18:39:51 INFO dspy.teleprompt.gepa.gepa: Iteration 33: Proposed new text for urgency_module.predict: Task: Read the provided message and determine the urgency.

Input format:
- You will receive a single “message” text (may include a subject line and body). Analyze the content for urgency cues.

Context/domain:
- Messages typically relate to facility management and services for a provider like ProCare Facility Solutions.
- Common topics: facility operations, space utilization, security, sustainability, HVAC systems, maintenance, and cleaning services.
- Senders may be residential or commercial clients and may reference residents, tenants, property operations, or prior support interactions.

Output format:
- Provide exactly two fields, in this order, no extra text or formatting:
reasoning: <1–3 concise sentences explaining the key cues that determine urgency>
urgency: <one of: low | medium | high>

Decision rules for urgency:
- HIGH:
  - Clear or implied immediate risk to safe

Average Metric: 2.53 / 3 (84.4%): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:05<00:00,  1.98s/it]

2025/08/12 18:39:58 INFO dspy.evaluate.evaluate: Average Metric: 2.533333333333333 / 3 (84.4%)


2025/08/12 18:40:28 INFO dspy.teleprompt.gepa.gepa: Iteration 34: Proposed new text for sentiment_module.predict: Task
- Read the provided message text and classify its overall sentiment as one of: positive, neutral, or negative.

Input format
- You will receive one field:
  - message: A string that may include a Subject line and an email-style body.

Output format
- Output only a single lowercase label: positive, neutral, or negative.
- Do not include any additional text, punctuation, or reasoning.

What to evaluate
- Focus on the overall emotional tone expressed about the service/interaction (e.g., satisfaction vs. dissatisfaction), not the message’s functional purpose (e.g., making a request), logistics, or formalities.

General decision rule
1) If the message is primarily praise/thanks expressing clear satisfaction or enthusiasm about the service/interaction, label positive.
2) Else if it expresses clear dissatisfaction, complaints, frustration, anger, fear, or disappointment about

## Now, let's evaluate the optimized program

In [51]:
evaluate(optimized_program)

Average Metric: 58.77 / 68 (86.4%): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [00:00<00:00, 527.68it/s]

2025/08/12 18:42:31 INFO dspy.evaluate.evaluate: Average Metric: 58.766666666666666 / 68 (86.4%)


,message,answer,urgency,sentiment,categories,metric
0,"Hey ProCare Support Team, Hope you all are doing great! My name is...","{""categories"": {""routine_maintenance_requests"": false, ""customer_f...",low,positive,[sustainability_and_environmental_practices],✔️ [1.000]
1,"Hey ProCare Team, Hope you’re all doing well! My name’s Jake, and ...","{""categories"": {""routine_maintenance_requests"": true, ""customer_fe...",medium,positive,[quality_and_safety_concerns],✔️ [0.933]
2,"Subject: Assistance Needed for HVAC Maintenance Hi [Receiver], I h...","{""categories"": {""routine_maintenance_requests"": true, ""customer_fe...",medium,neutral,"[routine_maintenance_requests, cleaning_services_scheduling]",✔️ [0.967]
3,Subject: A Green Inquiry from a Bill Maher Enthusiast Hey ProCare ...,"{""categories"": {""routine_maintenance_requests"": false, ""customer_f...",low,positive,[sustainability_and_environmental_practices],✔️ [1.000]
4,Subject: Inquiry on Sustainability Practices Dear ProCare Facility...,"{""categories"": {""routine_maintenance_requests"": false, ""customer_f...",low,neutral,[general_inquiries],✔️ [0.933]
...,...,...,...,...,...,...
63,Subject: Inquiry About Your Eco-Friendly Practices Dear ProCare Fa...,"{""categories"": {""routine_maintenance_requests"": false, ""customer_f...",low,neutral,"[general_inquiries, sustainability_and_environmental_practices]",✔️ [0.967]
64,Subject: Assistance Needed for Facility Management Issue Dear ProC...,"{""categories"": {""routine_maintenance_requests"": false, ""customer_f...",medium,neutral,[facility_management_issues],✔️ [0.667]
65,"Subject: Request for Training and Support Hi ProCare Support Team,...","{""categories"": {""routine_maintenance_requests"": false, ""customer_f...",low,neutral,[training_and_support_requests],✔️ [0.667]
66,Subject: Concerns About Studio Maintenance and Rent Increase Dear ...,"{""categories"": {""routine_maintenance_requests"": true, ""customer_fe...",medium,neutral,"[facility_management_issues, quality_and_safety_concerns]",✔️ [0.933]


EvaluationResult(score=86.42, results=<list of 68 results>)

GEPA was able to optimize the base program **from 74.41% score to 81.27% score** in just 11 iterations. With higher budget, the optimized program's score can go as high as **89.41%**

### Let's print the prompts that GEPA discovered

In [52]:
for name, pred in optimized_program.named_predictors():
    print("================================")
    print(f"Predictor: {name}")
    print("================================")
    print("Prompt:")
    print(pred.signature.instructions)
    print("*********************************")

Predictor: urgency_module.predict
Prompt:
Task: Read the provided message and determine the urgency.

Context/domain:
- Messages typically relate to facility management and services (e.g., facility operations, space utilization, security, sustainability, HVAC systems, maintenance, cleaning services) for a provider like ProCare Facility Solutions.
- Senders may be residential or commercial clients and may reference residents, tenants, property operations, or prior support interactions.

Output format:
- Provide exactly two fields, in this order, no extra text or formatting:
reasoning: <1–3 concise sentences explaining the key cues that determine urgency>
urgency: <one of: low | medium | high>

Urgency levels and decision rules:
- HIGH:
  - Clear or implied immediate risk to safety/security or major operational impact.
  - Explicit urgency signals (e.g., “Urgent,” “Immediate attention required,” “ASAP,” “critical,” “escalating”).
  - Severe dissatisfaction with demand for immediate corre